In [7]:
from typing import NamedTuple
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import storage
import os
import pandas as pd
import datetime
from kfp.v2 import dsl, compiler
from kfp.v2.google.client import AIPlatformClient
import google_cloud_pipeline_components.aiplatform as gcc_ai
from typing import NamedTuple
import kfp
from kfp import dsl
import kfp.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import storage
from io import StringIO
import pandas as pd
from google.cloud import storage
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [1]:
USER_FLAG = "--user"

In [2]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.10
google_cloud_pipeline_components version: 0.1.1


In [3]:
import os
PROJECT_ID = ""
# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  kedro-kubeflow-334417


In [5]:
BUCKET_NAME="gs://diab-gsbucket"
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"
PIPELINE_NAME = "diabetes_pipeline"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://diab-gsbucket/pipeline_root/'

In [8]:
@component(base_image="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest",
            output_component_file="test_train_data_split_component.yaml")
def data_component(bucket: str, path: str, data_file: str, value: float, marker: int) -> int:
    import kfp
    import pandas as pd
    from kfp.v2.google.client import AIPlatformClient
    data = pd.read_csv(f'gs://{bucket}/{path}/{data_file}.csv', sep=",")
    train_data = data.drop('Outcome',axis=1)
    test_data = data['Outcome']
    
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_data, test_data, test_size = value, random_state=42)
    
    # upload train/test data innto cloud storage
    client = storage.Client()
    bucket = client.get_bucket(f'{bucket}')
    
    # 1. X_train
    bucket.blob('data/X_train.csv').upload_from_string(X_train.to_csv(), 'text/csv')
    xtrain = f"gs://{bucket}/{path}/X_train.csv"
    # 2. X_test
    bucket.blob('data/X_test.csv').upload_from_string(X_test.to_csv(), 'text/csv')
    xtest = f"gs://{bucket}/{path}/X_test.csv"
    # 3. y_train
    bucket.blob('data/y_train.csv').upload_from_string(y_train.to_csv(), 'text/csv')
    ytrain = f"gs://{bucket}/{path}/y_train.csv"
    # 4. y_test
    bucket.blob('data/y_test.csv').upload_from_string(y_test.to_csv(), 'text/csv')
    ytest = f"gs://{bucket}/{path}/y_test.csv"
    
    marker = 1
    return marker

In [9]:
@component(base_image="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest",
            output_component_file="model_component.yaml")
def model_component(bucket:str, path:str, xtrain:str, ytrain:str, xtest:str, ytest:str, marker: int) -> float:
    import pandas as pd    
    import sklearn
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    with dsl.Condition(marker == 1):
        # read X_train and y_train from cloud storage
        X_train = pd.read_csv(f'gs://{bucket}/{path}/{xtrain}.csv', sep=",")
        y_train = pd.read_csv(f'gs://{bucket}/{path}/{ytrain}.csv', sep=",")
        # train model
        rc = RandomForestClassifier(max_depth=2, random_state=0)
        model = rc.fit(X_train, y_train)
    
        # read X_test and y_test from cloud storage
        X_test = pd.read_csv(f'gs://{bucket}/{path}/{xtest}.csv', sep=",")
        y_test = pd.read_csv(f'gs://{bucket}/{path}/{ytest}.csv', sep=",")
        # validate model
        y_pred = rc.predict(X_test)
        accuracy = accuracy_score(y_test,y_pred)
        
    return accuracy

In [10]:
@component(base_image="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest",
            output_component_file="true_decision_component.yaml")
def true_component(accuracy:float) -> None:
    print(f'Yes!! {accuracy} is the Accuracy and its greater than the threshold')

In [11]:
@component(base_image="us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest",
            output_component_file="false_decision_component.yaml")
def false_component(accuracy:float) -> None:
    print(f'No. {accuracy} is the Accuracy and its smaller than the threshold')

In [12]:
@kfp.dsl.pipeline(name = "diabetes-pipeline",
                  pipeline_root = PIPELINE_ROOT)
def diab_pipeline(
    display_name: str=f"{PIPELINE_NAME}-2021MMDD-nn",
    project: str = PROJECT_ID,
    gcp_region: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    marker: int = 0,
    test_train_split_ratio: float = 0.3,
    accuracy_threshold: float = 0.5
) -> None:
        
    # initiating data component
    data_op = data_component(BUCKET_NAME, "data", "diabetes", test_train_split_ratio, marker)
    # initiating model component
    model_op = model_component(BUCKET_NAME, "data", "X_train", "y_train", "X_test", "y_test", data_op.output)
    
    with dsl.Condition(model_op.output >= accuracy_threshold, name="accuracy>=50"):
        true_component(model_op.output)
    with dsl.Condition(model_op.output < accuracy_threshold, name="accuracy<50"):
        false_component(model_op.output)
    '''
    # decision node
    with kfp.dsl.Condition(acc_metric.output >= accuracy_threshold, name="accuracy check"):
        print(f"{acc_metric} is accuracy and is greater than its threshold")
    '''
    

In [13]:
compiler.Compiler().compile(
    pipeline_func=diab_pipeline, package_path="diab_pipeline.json"
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1266: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


In [14]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


In [15]:
response = api_client.create_run_from_job_spec(
    "diab_pipeline.json", pipeline_root=PIPELINE_ROOT,
)